In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd

In [2]:
# Choose the executable path to driver for Mac
#executable_path = {'executable_path': 'chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Choose the executable path to driver for Windows
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
#Goto to the NASA url
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

In [5]:
# get the html
nasa_html = browser.html

# parse html in bs
soup = bs(nasa_html, 'html.parser')


# grab the first instance of news
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text


print(news_title)
print(news_p)

NASA Wins Two Emmy Awards for Interactive Mission Coverage
NASA-JPL's coverage of the Mars InSight landing earns one of the two wins, making this the NASA center's second Emmy.


In [6]:
#goto JPL
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [7]:
#get the html
jpl_html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(jpl_html, 'html.parser')

# Retrieve background-image url from style tag. The image url is a relative path with some panethses and quotes.
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Website Url 
main_url = 'https://www.jpl.nasa.gov'

# Concatenate website url with scrapped route
featured_image_url = main_url + featured_image_url

In [8]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA22893-1920x1200.jpg'

In [9]:
#Mars weather on twitter. This might be the first time I've intentially visited twitter.
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [10]:
weather_html=browser.html

soup = bs(weather_html, 'html.parser')

#get the tweets, pretty easy they must want this to happen
tweets = soup.find_all('div',class_='js-tweet-text-container')

#find the first entry with the weather data. a/o the running this the last weather update was 9/15
#they seem to start w/ Insight sol
for tweet in tweets: 
    weather_tweet = tweet.find('p').text
    if 'InSight sol' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 285 (2019-09-15) low -100.8ºC (-149.5ºF) high -27.1ºC (-16.8ºF)
winds from the SE at 5.4 m/s (12.2 mph) gusting to 21.3 m/s (47.6 mph)
pressure at 7.40 hPapic.twitter.com/HtCsngFytC


In [11]:
# Visit Mars facts url 
facts_url = 'http://space-facts.com/mars/'

mars_facts_df = pd.read_html(facts_url)[0]
mars_facts_df = mars_facts_df.rename(index=str, columns={0: "Description", 1: "Value"})
mars_facts_html = mars_facts_df.to_html(index='False',index_names='False')
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Temperature:</td>\n      <td>-153 to 20 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [12]:
#Hemisphere pics
hemi_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

In [13]:
#start a dict list for wrangling the images
hemi_urls = []

# return lisr of all the image links
links = browser.find_by_css("a.product-item h3")
#loop through the image links
for item in range(len(links)):
    hemis = {}
    #this find by css thing is easier on this page
    browser.find_by_css("a.product-item h3")[item].click()
    #read the docs for the browser class, maybe go back to the previous searches to clean them up too
    #this returns 
    sample_element = browser.find_link_by_text("Sample").first
    #get the title, i.e. the hemisphere
    hemis["title"] = browser.find_by_css("h2.title").text
    
    #get the image url
    hemis["img_url"] = sample_element["href"]
    #add the image urls to a dict
    hemi_urls.append(hemis)
    #click back to go to the next image
    browser.back()

In [14]:
hemi_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]